# Songs Recommender System    

Recommender systems are a popular class of information filtering system. The goal of such systems is to predict the preference an user would give to an item/ service and thus "recommend" them with those relevant items. Recommender systems are known to improve user experience on many webportals, especially the ones which involve lots of social interaction or shopping. Here, we explore **content based recommenders** that leverage:   
1. Popularity of items   
2. Matrix Factorization (Singular Value Decomposition)

<strong>Dataset:</strong>    
We use the "Taste profile subset" dataset that is auxillary to the popular million songs dataset available at: [The Million Song Dataset](https://labrosa.ee.columbia.edu/millionsong/)
It is a freely-available collection of audio features and metadata for a million contemporary popular music tracks. The purpose of choosing such a large dataset is to build and test <font color="blue">recommender systems at scale</font>. Although the entire dataset is of 280 GB, we deal only with **3 GB** of available sample data which still exhibits characteristics of a large dataset while being moderately computationally intensive to model. 

In [31]:
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt;
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from google.cloud import bigquery
from collections import defaultdict
import os

## Setting up BigQuery

Note: You will have to change the path to the key in your computer.

In [2]:
import os

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/gretalerer/Desktop/music-recommendation/chatbots-project-417113-53eaab96a4be.json"


In [3]:
client = bigquery.Client(location="US") 


### 1. Import Data from log files and basic exploration 
Since, the overall dataset has more than a million unique users and records about 384,000 songs, we would work with a subset (10000 records) of data

In [7]:
query = """
SELECT * FROM `chatbots-project-417113.raw_data.train_triplets` LIMIT 10000
"""
dfsongs = client.query(query).to_dataframe()

Let us inspect some of the imported records and other characteristics of data before we proceed further

In [8]:
dfsongs.head(5)

,user,song,play_count
0,b493eda768d25cfb48dec2567b0826cdb9f6eef4,SOCBSZW12AB01891C1,256
1,d50046ceb4db11dc162649a0da27733ff5194b47,SOAAAGQ12A8C1420C8,1
2,1e93ab57208a575ad87872ff3a25c1743632d0a1,SOAAAGQ12A8C1420C8,1
3,a0846981ab7c3d06cf1d966e7109774e306eb61a,SOAAAGQ12A8C1420C8,1
4,6e8d4e7d986e077d334da41638e63b9030710141,SOAAAGQ12A8C1420C8,1


### Power-Law in real world systems
   Next, we are interested to know the number of unique users that we need to consider to build an useful recommender system. However, in most real world web applications, we see that only a few number of users consume a majority of the services while most other users use the services very rarely. In such a scenario, we need to take into account only those frequent song listeners to build a significant recommender.   

<strong>Efficiently summarizing 3.5 million records:</strong>    
    So, to decide the percentage of such users, we need to group the records by user id. However, it would be inefficient to use pandas grouping functions over 3.5 million records. Hence, we would iterate over the rows of records and summarize the user count.   

### 2. Summarizing records by users and songs:

We first summarize the records by users and then by songs.    
<strong>2.1 Users - play count summary </strong>

In [12]:
usersDict = defaultdict(int)

for index, row in dfsongs.iterrows():
        user = row['user']
        play_count = row['play_count']
        usersDict[user] += play_count
        
userPlayCountList = [{'user': u, 'play_count': p} for u, p in usersDict.items()]
dfSongsPlayCount = pd.DataFrame(userPlayCountList)
dfSongsPlayCount = dfSongsPlayCount.sort_values(by = 'play_count', ascending = False)
#dfSongsPlayCount.to_csv("songs_play_count.csv", index=None)

As seen earlier, now we arbitrarily consider top 40% of the songs play count and need to determine the number of users accounting for this percentage. So, we consider only top 100,000 users ranked according to number of songs listened by them.

In [13]:
totalPlayCount = sum(dfSongsPlayCount.play_count)
(float(dfSongsPlayCount.head(n=100000).play_count.sum())/totalPlayCount)*100
dfUserSubset = dfSongsPlayCount.head(n=100000)

<strong>2.2 Songs - play count summary:</strong>

In [14]:
from collections import defaultdict

# Initialize a default dictionary to store play counts by song
songsDict = defaultdict(int)

# Iterate over the DataFrame records
for index, row in dfsongs.iterrows():
    # Fetch song id and play count from the DataFrame
    song = row['song'] 
    play_count = row['play_count']  

    # Update song play count for a song
    songsDict[song] += play_count

# Convert the default dictionary to a list of dictionaries
songPlayCountList = [{'song': s, 'play_count': p} for s, p in songsDict.items()]

# Convert the list of dictionaries to a DataFrame
dfSongsPlaySummary = pd.DataFrame(songPlayCountList)

# Arrange the songs played records in descending order to facilitate further analysis
dfSongsPlaySummary = dfSongsPlaySummary.sort_values(by='play_count', ascending=False)


In [15]:
(float(dfSongsPlaySummary.head(n=30000).play_count.sum())/totalPlayCount)*100

100.0

<font color="blue">Observation:</font>     
We observe that only top 30% of the songs are listened on almost 80% of the occassions and less than 40% of the users listen to almost all the songs.   
So, we subset the songs and users summary datasets accordingly.   

In [16]:
dfSongPlayCountSubset = dfSongsPlaySummary.head(n=30000)

Now, we have obtained top 30% of the popular songs that users listen. Next, we need to identify these songs from the original dataset that contained play counts corresponding to each user for these songs.

In [18]:
dfSongsTopUsers = dfsongs[dfsongs['user'].isin(dfUserSubset['user'])]

dfSongsTopSongs = dfSongsTopUsers[dfSongsTopUsers['song'].isin(dfSongPlayCountSubset['song'])]


In [19]:
#Find the number of records in this subset
dfSongsTopSongs.shape

(10000, 3)

So, we have obtained about 1 Million records related to top 30% of the popular songs. We also delete the unwanted subset dataframes to free up the memory

In [22]:
del dfsongs 
del dfSongsTopUsers

Looking at the few records from the newly obtained subset of popular songs

In [23]:
dfSongsTopSongs.head(5)

,user,song,play_count
0,b493eda768d25cfb48dec2567b0826cdb9f6eef4,SOCBSZW12AB01891C1,256
1,d50046ceb4db11dc162649a0da27733ff5194b47,SOAAAGQ12A8C1420C8,1
2,1e93ab57208a575ad87872ff3a25c1743632d0a1,SOAAAGQ12A8C1420C8,1
3,a0846981ab7c3d06cf1d966e7109774e306eb61a,SOAAAGQ12A8C1420C8,1
4,6e8d4e7d986e077d334da41638e63b9030710141,SOAAAGQ12A8C1420C8,1


### 3. Enhance Dataset with Song Track Details
Though we have obtained a subset of popular songs, it lacks song titles, artist names and other such details. These details can be retrived from a available supporting dataset of metadata in SQLite format at [Million Songs Metdata](https://labrosa.ee.columbia.edu/millionsong/pages/getting-dataset#subset)

In [25]:
query = """
SELECT * FROM `chatbots-project-417113.raw_data.track_metadata_db` 
"""
dfTrackMetadata = client.query(query).to_dataframe()
dfTrackMetadata_sub = dfTrackMetadata[dfTrackMetadata.song_id.isin(dfSongPlayCountSubset.song)]


In [14]:
#Retrieve details of song tracks from the metadata file
dfTrackMetadata = pd.read_sql(con=conn, sql='select * from songs')
dfTrackMetadata_sub = dfTrackMetadata[dfTrackMetadata.song_id.isin(list(dfSongPlayCountSubset.song))]

Observe the song details dataset

In [26]:
dfTrackMetadata_sub.head(3)

,track_id,title,song_id,release,artist_id,artist_mbid,artist_name,duration,artist_familiarity,artist_hotttnesss,year,track_7digitalid,shs_perf,shs_work
131,TRMOLXY128F9333968,Amnistía,SOASWIB12AB0181858,Poetics,AR00FVC1187FB5BE3E,ecd6ec3e-18d3-45e3-91a0-06e7beeb5f0a,Panda,120.47628,0.768987,0.550031,0,7057023,-1,0
144,TRQYZHP12903CCBDCA,Ilasha,SOASCSI12AB0188BBC,La Revancha Del Principe Charro,AR00FVC1187FB5BE3E,ecd6ec3e-18d3-45e3-91a0-06e7beeb5f0a,Panda,188.81261,0.768987,0.550031,0,6519290,-1,0
185,TRGCRGP128F4277FA3,Pop Quiz (Multiple Choice),SOANGRR12A8C13FD86,Warning Shots,AR00JIO1187B9A5A15,5251b5a0-3e3b-4d07-a152-585009575310,Saigon,272.69179,0.720672,0.421353,2004,2908418,-1,0


### 4. Preprocessing the data 
Now that we have all the relevant information, we need to aggregate the song details and user Ids and remove the unwanted columns from the final cleaned table

In [27]:
del(dfTrackMetadata_sub['track_id'])
del(dfTrackMetadata_sub['artist_mbid'])
#Drop duplicate songs ids, as we are concerned with only unique songs
dfTrackMetadata_sub.drop_duplicates(['song_id'], inplace=True)
#Join the earlier obtained top popular songs dataset and this metadata dataframe on song id
dfPopularSongMetaDataMerged = pd.merge(dfSongsTopSongs, dfTrackMetadata_sub, how="left", 
                                      left_on="song", right_on = "song_id")
#Keep only relevant columns
dfPopularSongMetaDataMerged = dfPopularSongMetaDataMerged[['user', 'song', 'title',
                                                           'play_count', 'release', 'artist_name', 'year']]
dfPopularSongMetaDataMerged.head(3)

/var/folders/sw/6q2vf2dx0r502xm9jxzvdxzc0000gn/T/ipykernel_90251/3474166419.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfTrackMetadata_sub.drop_duplicates(['song_id'], inplace=True)


,user,song,title,play_count,release,artist_name,year
0,b493eda768d25cfb48dec2567b0826cdb9f6eef4,SOCBSZW12AB01891C1,XRDS,256,Ritual Noise,Covenant,0
1,d50046ceb4db11dc162649a0da27733ff5194b47,SOAAAGQ12A8C1420C8,Orgelblut,1,Dolores,Bohren & Der Club Of Gore,2008
2,1e93ab57208a575ad87872ff3a25c1743632d0a1,SOAAAGQ12A8C1420C8,Orgelblut,1,Dolores,Bohren & Der Club Of Gore,2008


In [28]:
dfPopularSongMetaDataMerged.to_csv("popular_songs_metadata.csv", index=False)

In [ ]:
# List of intermediary CSV file paths to delete
files_to_delete = [
    "songs_play_count.csv",
    "songs_summary.csv",
    "popular_songs_metadata.csv",
    # Add any other file paths here
]

# Loop through the list and delete each file
for file_path in files_to_delete:
    try:
        os.remove(file_path)
        print(f"Deleted {file_path}")
    except OSError as e:
        print(f"Error deleting {file_path}: {e.strerror}")
